<div style="text-align: right">INFO 6105 Data Science Eng Methods and Tools, Take-home Midterm</div>
<div style="text-align: right">Dino Konstantopoulos, 14 March 2023</div>

# So, is it the car or is it the driver, in ten parts

Well, I got this idea for your midterm when one of you asked the question in class "*so how do we model F1 with a Bayesian approach*"?

<br />
<center>
<img src="ipynb.images/f1-2022.jpg" width=1000 />
</center>

The big question in F1 is [is it the car or is it the driver](https://www.total-motorsport.com/f1-car-or-driver-more-important/). We want to begin to answer this question with modern bayesian statistics.

We want to infer the hidden parameters (how talented a F1 driver is, how strong a F1 constructor is - these parameters are hidden to us) that are driving the data we observe (points), and be able to use these to predict ***future*** points by building a **model** for each car.

We want to start with some **prior** distributions, and use observations (past results) to refine our priors into **posteriors**. Points are a noisy measurement of car strength, so quantifying car strength **uncertainty** is important. It will help us determine where to bet less (cars we're weaker at predicting), and cars to bet more on (cars where uncertainty is minimal).

We will use a Poisson count model to model F1 car points, since [F1 points](https://en.wikipedia.org/wiki/Formula_One_racing) awarded, exclusively for positions 1 through 10, are integer points.

>*Points are awarded to drivers and teams based on where they finish in a race. The winner receives 25 points, the second-place finisher 18 points, with 15, 12, 10, 8, 6, 4, 2 and 1 points for positions 3 through 10, respectively*.

I did a bit of litterature research and found this interesting [paper](https://arxiv.org/pdf/2203.08489.pdf), which underscores that in a race, it's mostly about the car, rather than the driver.

>**Note**: We will simplify the paper's approach somewhat, but if you read the paper carefully, you will realize what a long trip we have taken together already and how much you've learned. This is an advanced research paper in data science, so if you feel somewhat comfortable reading it, well done student!

Let's see if this is true.

First, a bit about pymc3...

# About pymc3
`PyMC3` has been renamed to `PyMC`. PyMC3 version 3.x will stay under the current name to not break production systems

Here's what to remember:

- PyMC3 3.x: The version we all love and use, using the `theano-pymc` tensor
- PyMC version 4, with the latest version being 5.1.1: The successor to PyMC3, (mostly) API compatible, and main focus going forward.
- (PyMC4: A discontinued experiment with new API on the TensorFlow backend).

`Theano` is a computational graph library, compiling complex mathemtical operations into one graph that can be optimized (we experimented with similar optimizations when we contrasted lazy list comprehensions and generators versus eager list comprehensions and functions). Theano was absorbed into tensorflow, at which point the Theano authors (Montral university) discontinued it. The pymc3 authors then did what everyone would do: Upgrade pymc3 to use tensorflow, but they failed (thus discontinued `pymc4`). After rediscovering the power of Theano, the pymc3 authors forked Theano to `Theano-PyMC` (PyMC3 now relies on this fork), and then reforked to `aesara`, and then reforked to `pytensor`.

In v3, pymc3 was not really using the full power of Theano. While calling e.g. `x = pm.Normal('x')` created a `theano.TensorVariable` which can be manipulated further, like inputting it into another random variable, a full-fledged graph was not being built.

[Here's](https://gist.github.com/twiecki/0219d3fa059776b90af0f08b5452e346) more detail about all these shenanigans.

If you can run pymc3 with the theano tensor on your laptop (run first cell in this notebook), great, keep using it. Otherwise, please create a new environment and install pymc version 5.1.1 based on python 3.10:
```
conda install nb_conda_kernels
conda create -n pymc5 python=3.10
conda activate pymc5
conda install -c conda-forge pymc==5.1.1
pip install seaborn
conda install ipykernel
(restart anaconda kernel)
```

then open the notebook, go to kernels menu, and switch to your new environment by selecting the pymc5 kernel, and run the second cell in this notebook instead of the first cell. ***DO NOT*** run both cells. Pick the one you will use.

Run the cell below if you're using the `theano` tensor. If it works, keep using the theano tensor and ***do not run the second cell below***.

If you don't have `theano`, another option would be to stick with `pymc3` and replace `tt.exp` with `pm.math.exp` and `tt.mean` with `pm.math.sum(..)/num_teams` in setting up the simulation. So, you have a number of options.

In [5]:
import numpy as np
import pandas as pd
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO
import pymc as pm, theano.tensor as tt
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
import seaborn as sns
%matplotlib inline

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


ModuleNotFoundError: No module named 'cachetools'

If the cell above failed, then you should probably be using `pytensor` instead of `theano`.
```
conda install nb_conda_kernels
conda create -n pymc5 python=3.10
conda activate pymc5
pip install pymc==5.1.1
pip install seaborn
conda install ipykernel
(restart anaconda kernel)
```

Then run the cell below, and keep on truckin'!

In [1]:
import numpy as np
import pandas as pd
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO
import pymc3 as pm

# import pytensor
# from pytensor import tensor as tt

import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
import seaborn as sns
%matplotlib inline

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


>***Note***: If you get the following output, then you're probably hosed. See your TAs!
```
WARN: Could not locate executable g77
WARN: Could not locate executable f77
WARN: Could not locate executable ifort
WARN: Could not locate executable ifl
WARN: Could not locate executable f90
WARN: Could not locate executable efl
WARN: Could not locate executable gfortran
WARN: Could not locate executable f95
WARN: Could not locate executable g95
WARN: Could not locate executable efort
WARN: Could not locate executable efc
WARN: Could not locate executable flang
WARN: don't know how to compile Fortran code on platform 'nt'
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
```

# 1. The data for F1 season 2022
I found it [here](https://github.com/toUpperCase78/formula1-datasets)!

In [ ]:
df_races = pd.read_csv('data/Formula1_2022season_raceResults.csv')

In [ ]:
df_races

# 2. EDA
Do some data healing.

# 3. Points
Build a dictionary that represents the race points won by a driver as a function of race finishing position.

Then, build a pandas dataframe called `df_races_positions` with the same index as the `df_races` dataframe, but with only the following columns: `Position`, `Driver`, `Team`, `Points`.

Then, build the following dataframes (in full) representing driver + team and total points for the season, and team and points for the season:
```
	Position	Points
Driver	Team		
Max Verstappen	Red Bull Racing RBPT	59	428
Charles Leclerc	Ferrari	63	288
Sergio Perez	Red Bull Racing RBPT	79	288
George Russell	Mercedes	92	259
Lewis Hamilton	Mercedes	113	231
...
```

```
Position	Points
Team		
Red Bull Racing RBPT	138	716
Ferrari	119	514
Mercedes	205	490
Alpine Renault	311	170
...
```

Then look at the mean and variance of driver points. You should find that mean and variance are not super different in range, so it makes sense to use a Poisson distribution to model Points.

Plot a bar chart with Season 2022 Standings for each driver.

Index the drivers and the teams, and build a python list called `drivers_observed` that lists all the drivers by index in the order of the `df_races` dataframe.

Do the same with Teams (also called *constructors*) in a python list called `constructors_observed`.

Finally, build a python list called `points_observed` that represents the points awarded to each car finishing in the order of the `df_races` dataframe.

These three lists will figure prominently in our simulation.

In [2]:
# Create dictionary to map race finishing position to race points
race_points_dict = {1: 25, 2: 18, 3: 15, 4: 12, 5: 10, 6: 8, 7: 6, 8: 4, 9: 2, 10: 1}

# Create a new dataframe with only the relevant columns
df_races_positions = df_races[['Position', 'Driver', 'Team']]

# Add a new column to the dataframe that maps finishing position to race points
df_races_positions['Points'] = df_races_positions['Position'].map(race_points_dict)

# Create a pivot table to aggregate points by driver and team
df_driver_team_points = pd.pivot_table(df_races_positions, values='Points', index=['Driver', 'Team'], aggfunc=np.sum)

# Create a pivot table to aggregate points by team
df_team_points = pd.pivot_table(df_races_positions, values='Points', index=['Team'], aggfunc=np.sum)

# Sort the pivot tables by points
df_driver_team_points = df_driver_team_points.sort_values(by=['Points'], ascending=False)
df_team_points = df_team_points.sort_values(by=['Points'], ascending=False)

# Calculate the mean and variance of driver points
driver_points_mean = df_driver_team_points['Points'].mean()
driver_points_variance = df_driver_team_points['Points'].var()

# Plot a bar chart with Season 2022 Standings for each driver
df_driver_team_points.plot(kind='bar', y='Points', legend=False)
plt.ylabel('Points')
plt.title('Season 2022 Standings')
plt.show()

# Get a list of all observed drivers and constructors
drivers_observed = df_races_positions['Driver'].unique()
constructors_observed = df_races_positions['Team'].unique()

# Get a list of points awarded to each car finishing in the order of the df_races dataframe
points_observed = df_races_positions['Points'].tolist()


NameError: name 'df_races' is not defined

## Building a Bayesian Power Model

We know that a **count outcome** is modelled as a **Poisson distribution**, unless mean and standard deviation really diverge, in which case we use the **negative binomial**. 

Modeling car strength is usually accomplished with a vector of points scored $y$ as a Poisson distributions: $(y_i\;|\;θ_j) \propto Poisson(θ_j)$ where the $\theta_j$ parameters represent the Poisson **expectation** for the car finishing the race.

We will follow the approach of the research paper, simplifying it a bit, and model the point expectation $\theta$ of each car as an exponential of *two* independent variables: a drivers variable **drv**, and a constructor variable **csr**, times a constant factor k:

$$θ_c = k_c * e^{\text{drv}_c} * e^{\text{csr}_c}$$

Instead of 4 (in the paper) where the other two parameters are longer-term effects related to the *experience* of each team *across* seasons. 

>**Note**: Exponential, professor? I thought we were building **linear models**!

Yes, we use an exponential to increase the effect of each indendent variable,  but we will use $log(\theta)$ to simplify to a linear model!

Parameters are initially modeled with a log-linear random effect model, a standard procedure in sports analytics.

$$\log(θ) = log(k) + \text{drv}_c + \text{csr}_c $$

>**Note**: Wait, where's the $x$? There really ***is no $x$***! So we don't really have a linear model, here. Instead, we have an **additive model**, and we create two *new* columns: $\text{att}_g$ and $\text{def}_g$.

Let's assume results are determined jointly by a ***driver*** and ***constructor*** ability for each event, an aggregation of the talent of drivers and mechanics, represented by parameters `drv` and `csr`, respectively.

For each car race result $c = 1, \cdots, C$, car-specific `drv` and `csr` effects will be modelled by a common normal distribution:
$drv_c \propto \text{Normal}(μ_{drv},τ_{drv})$ and $csr_c \propto \text{Normal}(μ_{csr}, τ_{csr})$.

We will initialize $μ_{drv}$ and $μ_{csr}$ to 0, which is a common action in mcmc sims for modeling likelihood parameter pdfs.

As for $\sigma_{drv}$ and $\sigma_{csr}$, we will initialize them to... *another pdf*! This gives the mcmc engine more leeway in the simulation. We will initialize them to half-Student-t distributions. The half-Student-t parameters will be set to match the data's.

We will build our model in `PyMC3`, specifying global parameters, car-specific parameters, and the likelihood function.

Ok, so the data we want to consider are the points awarded for each car in `df_races`.

# 4. Priors
The number of points scored for the entire season, divided by the number of races is a good indicator of both ***driver talent*** and ***constructor know-how***.

So, these will be our priors for the `drv` and `csr` parameters for each car: 

Take the floor of the mean number of points for 2022 per driver, and then its logarithm. It should look like this:
```
Driver
Max Verstappen      6.059123
Charles Leclerc     5.662960
Sergio Perez        5.662960
....
```

Replace infinities with zeros.

Build a python list called `drv_starting_points` that represents these values, in the order of your driver index.

Take the floor of the mean number of points for 2022 per constructor, and then its logarithm. 

Build a python list called `csr_starting_points` that represents these values, in the order of your team index.

These will be our priors for our sim,

# 5. Likelihood function and model parameters
Now let's specity our **likelihood function** in `pymc3`. 

A bit of theory:

You should know that whenever we have a small number of data point (about 30 or less), the [t-distribution](https://en.wikipedia.org/wiki/Student%27s_t-distribution) gives better results than the normal distribution. 

If you are modeling the distribution of a *large* population, then use the **normal distribution**. If you are modeling the mean of the population when the population size is small, better to use the **student-t distribution**.

The student-t distribution has higher kurtosis (4th *moment*) than the normal distribution. That means that data from the t-distribution will have a tendency to appear closer or farther from the mean (it's like it says on your rear-view mirror: ***objects are closer than they appear***) than typical normal data, with a more sudden transition in between. In other words, the t-distribution is more *fat-tailed* and captures outliers ***better***: The probability of obtaining values far from the mean is larger than with the normal distribution.

As the number of degrees of freedom of data ($\nu$) increases, the t-distribution becomes closer to the normal distribution. The t-distribution is used a lot in [fintech](https://link.springer.com/referenceworkentry/10.1007%2F978-1-4419-7701-4_18). Financial data does tend to have higher kurtosis than the normal distribution, and exhibits wild jumps up or down  more frequently than what the normal distribution predicts.

The **half-t distribution** (also denoted **folded-t**) is derived from the t-distribution by taking the absolute values of variates. Since we have positive points, we don't want any negative signs.

But we're not going to use neither the gaussian nor the Student-t as our data likelihood. We are going to use the Poisson distribution, because we have *count* data.

Since we decided on a linear model for log$\theta$, we usually have a *slope-intercept* formula: $y = log(\theta) = mx + b$ where $m$ is the slope or the consistent change between $x$ and $y = log(\theta)$, and $b$ is the y-intercept (intercepts the y-axis at $y = b$). It is the expected mean value of $y$ when $x = 0$. Now, what is $x$? $x$ is the keys, and here we have discrete keys: drivers and constructors. Linear models are most often used with continuous keys. Here, since we have discrete keys, instead of a linear model we are going to use an **additive model**, where $log(\theta)$ can be expressed as the sum of two variables, each modeled as a stochastic distribution.

So here's the model and likelihood in `pymc3`, starting from the bottom of the cell below, to the top:

- The points scored by cars, our **data likelihoods**, will be modelled by Poisson distributions. The Poisson distribution has a single parameter: its **Expectation**, which we will model with a **bivariate linear model** with the two features being ***driver talent*** and ***constructor talent***. 

- Actually, we will compute with the **log** of the expectation, so to make the log of the expectation a linear model, we will assume that the expectation is actually an **exponential**, with an additive model for the exponent.

- Since driver talent and constructor know-how are relative concepts, we will model these strengths as ***car-specific strengths minus the average across all cars***. Note that we are using an additive relational. We also could have used a multiplicative relation whereby talents and know-how are the *ratio* of car-specific values, normalized by the average across all cars.

- The car-specific driver and constructor strenghts (talent and know-how) are parameters of our data likelihood, so we need to model them as probability density functions. What to pick? When we don't know, we pick **normal distributions**! How about their mean and standard deviation?

- Bayesian estimation usually assumes a 0 mean for these parameters. For the standard deviation (noise) we will use a half-normal. But to account for many possible outliers (liek Ferrari in 2022 that made a ton of [strategic mistakes](https://www.gpblog.com/en/news/163001/numerous-mistakes-by-ferrari-in-2022-show-it-was-time-for-binotto-exit.html)), we will use a **half-student-T** instead (noise is never negative, that's why a **half** profile). The student-T has 3 parameters itself, and here we select numbers: Pure guesses (mean 0, sd of 2.5, and a nu of 3).

- The intercept is completely unknown, so we select a **uniform distribution** (all values equally likely)

If you have no idea about the intercept, you could model it with a uniform distribution:
```
    intercept = pm.Flat('intercept') #flat pdf is uninformative - means we have no idea
```

Your car-specific model parameters, specifically one `(drvs_star, csrs_star)` tuple per car, could be modeled with normal distributions (you will need to replace the `?`s. Hint: use your priors!):
```
    drvs_star = pm.Normal("drvs_star", mu=0, sd=?, shape=?)
    csrs_star = pm.Normal("csrs_star", mu=0, sd=?, shape=?)
```

To allow samples of expressions to be saved, we need to wrap them in pymc3 Deterministic objects. Since we only care about *relative* strengths, we subtract from `drvs_star` and `csrs_star` the mean across drivers and constructors. We save these parameters for later exploration:
```
    drvs = pm.Deterministic('drvs', drvs_star - tt.mean(drvs_star))
    csrs = pm.Deterministic('csrs', csrs_star - tt.mean(csrs_star))
```

Assume an exponential search on the theta parameter:
```
    theta = tt.exp(intercept + drvs[drivers_observed] + csrs[constructors_observed])  
```

Assume a Poisson likelihood on the observed data (Points) with expectation `theta`:
```
    points = pm.Poisson('points', mu=theta, observed=points_observed)
```

Note that if you don't have `theano`, you could replace `tt.exp` with `pm.math.exp` and `tt.mean` with `pm.math.sum(..)/num_teams`. 

If you opted for pymc 5.1.1, you already have `pytensor` installed and you can replace `theano` with `pytensor`.

# 6. Simulation
Run the mcmc simulation and plot the traces for your model parameters (You may have to replace `pm.traceplot()` with `pm.plot_trace`).

```
    trace = pm.sample(5000, tune=1000, cores=1)
    pm.traceplot(trace)
```

Then, tabulate your model parameters:
```
import arviz as az
az.summary(trace, round_to=2)
```

# 7. Setting a hypothetical constraint
Let's set a constraint that Charles Leclerc (Ferrari) is a better driver than Max Verstappen (Red Bull). How much better does The Red Bull constructor need to be in order to justify this constraint?

Provide this answer in a percentage relative to Ferrari and Mercedes.

A `pm.Potential()` is an arbitrary factor that you can add to the model likelihood. In this example, by using `tt.switch()`, if the parameters satisfies you constraints in `tt.switch`, we add nothing to the model, otherwise we add `-inf` to the log-likelihood of the model, making it *impossible*.

Let's set a constraint that Charles Leclerc (Ferrari) is a better driver than Max Verstappen (Red Bull)
```
    pm.Potential('CL_>_MV', tt.switch( tt.mean(drvs_star[1]) > tt.mean(drvs_star[0]), 0., -np.inf))
```

You may replace the `tt.switch` API on setting a simulation constraint to `pm.math.switch`.

Then, run the sim again. Note that you may have to limit it to 1 chain due to numerical instabilities (simulation blows up with `Bad initial energy`). You will get a *lot* of divergences with the constraint, representing cases where the constraint is *not* fulfilled and thus sampled off.
```
`trace = pm.sample(5000, tune=1000, cores=1, chains=1)
```

Conclude.

# 8. Plots

From model posteriors, plot driver talent *and* constructor strength with *and* without the constraint above, *with* **credible intervals**, so you have uncertainty quantification of your estimate of cars' strengths.

For example, your plot for driver talent should look like this (I removed the names of the drivers on the x-axis):

<br />
<center>
<img src="ipynb.images/driver_strength_unknown.png" width=600 />
</center>

In Bayesian statistics, a **credible interval** is a range of values within which an unobserved parameter value falls with a particular subjective probability. It is an interval in the domain of a posterior probability distribution or a predictive distribution.

Credible intervals are analogous to **confidence intervals** in frequentist statistics we studied when we studied the t-test?

Bayesian intervals treat bounds as *fixed* and the estimated parameter as a *random variable*. In our experiment to determine the distribution of team strength `att`, if the subjective probability that `att`$_{germany}$ lies between 0.35 and 0.55 is 0.95, then $0.35\leq \text{att} \leq 0.55$ is a 95% credible interval.

Frequentist confidence intervals treat bounds as random variables and the parameter as a fixed value. A frequentist 95% confidence interval means that with a large number of repeated samples, 95% of such calculated confidence intervals would include the true value of the parameter. Aren't Bayesian credible intervals a much better metric? I think they are. However, Bayesian credible intervals do require knowledge of a situation-specific prior distribution, while frequentist confidence intervals do not.

The **highest posterior density interval** (HDI) is the interval which contains the required point estimate such that all points within the interval have a higher probability density than points outside the interval. The HDI is the narrowest interval containing the specified point estimate. Locating the HDI is usually accomplished using the Chen-Shao algorithm (Chen and Shao; 1999; Chen, Shao, and Ibrahim; 2000). For more info, this is the best [reference](https://cran.r-project.org/web/packages/HDInterval/HDInterval.pdf) i've found. In `pymc3`, you get it with `pm.stats.hpd`.

**Quantiles** are sets of values of a variate that divide a frequency distribution into equal groups, each containing the same fraction of the total population. The *Median* is an example of a quantile that separates the two halves of a group.

Then, plot posterior pdfs for all drivers and for all constructors.

## A note about Red Bull Racing
[Red Bull Racing](https://en.wikipedia.org/wiki/Red_Bull_Racing), also simply known as Red Bull or RBR and currently competing as Oracle Red Bull Racing, is one of two Formula One teams owned by conglomerate Austrian company Red Bull GmbH, the other being Scuderia AlphaTauri (previously Scuderia Toro Rosso).

Red Bull had **Cosworth** engines in 2005 and **Ferrari** engines in 2006. 

The team used engines supplied by **Renault** between 2007 and 2018 (from 2016 to 2018, the Renault engine was re-badged "TAG Heuer" following the breakdown in the relationship between Red Bull and Renault in 2015). During this partnership, they won four successive Drivers' and Constructors' Championship titles from 2010 to 2013, becoming the first Austrian team to win the title. 

The team began using **Honda** engines in 2019. The works Honda partnership culminated in 2021 following Red Bull driver Max Verstappen's World Drivers' Championship victory, with Verstappen also winning the championship in 2022. Honda left the sport officially after 2021, but will continue to supply complete engines from Japan to the team under Red Bull Powertrains branding until the end of 2025.

Starting in 2025, **Ford** will supply the engine, starting with a hybrid (gas/battery) block.

# 9. Validating your model with Posterior Predictive Checks

Comparing the data to **posterior predictions** of the model is called a **posterior predictive check** (PPC). When there appear to be systematic discrepancies that would be meaningful to address, we should consider expanding or changing the model so it may be a better description of the data.

You can read [here](https://www.sciencedirect.com/topics/mathematics/posterior-predictive-check) about Posterior Predictive Checks, and [here](https://www.sciencedirect.com/topics/mathematics/bayesian-model-comparison) about the sensitivity of the Bayes Factor to priors, whenc comparing different models (for example, one model that predicts planetary motions based on elliptical orbits around the sun, as posited by Galileo Galilei, and another that predicts planetary motions based on circular cycles and epicycles around the earth, as posited by the Catholic Church).

Let's do **posterior predictive checks**  in order to validate our model. The idea is to generate data from the model using parameters from draws from the posterior.

PPCs can help analyze the degree to which data generated from the model deviate from data generated from the true distribution. Visualizing the PPC is a great "***sense check***" of your model. In `pymc3`, it's done with `sample_ppc()` of your trace.

Posterior predictive checks are just like a simulation, with the optimal parameters of your model. 

It's like generating new futures from your model, so that you can generate, let's say 100 futures to see what the mean outcome will be.

For example: `ppc = pm.sample_ppc(trace, samples=500, model=f1_model, size=100)` will randomly draw 500 samples of parameters from the trace. Then, for each sample, it will draw 100 random variates from the Poisson distribution specified by the values of the parameters in that sample. 

If one does not specify the `samples` and `size` parameters, then it will draw one random variate from each trace point:
```
pp_trace = pm.sample_posterior_predictive(trace)
```

Recall that your original dataset had 440 observations.

So, with your model, generate 10,000 times season 2022 of F1. The observations order for each one of these simulated season is the same as our original data.

Gather all the simulated points and plot the HPD of Driver talent and constructor know-how after 10,000 simulated seasons. Verify that it matches your empirical results.

This is how you can verify that your model is correct.

That means our model produces results *statistically equivalent* to our observations. 

This verifies that our model is correct.

## The world where CL > MV
In this model, we used a constraint to force Charles Leclerc's talent to be higher than Max Verstappen. 

Generate a posterior predictive distribution to see who wins the season in this world. Keep in mind that in this world, Red Bull racing should have been found to be rated *much higher* than competitors in constructor know-how.

Is the hypothetical model where Charles Leclerc is a more talented driver than Max Verstappen still *possible*? Are posterior checks producing results in line with observations? Can you reject this model as an unlikely one?

# 10. Probability that Ferrari wins two consecutive races
Since we can produce simulated data that potentially covers all possible situations, it is straightforward now to extract conditional probabilities.

We need tons of data to cover edge cases, so let's produce *a million* simulated 2022 seasons.
```
pp_trace_f = pm.sample_ppc(trace, samples=1000000, model=...)
```

Then, compute the probability that Ferrari comes in 1/2 in any race in the 2022 season.

You should find it to be very small.

Indeed, Ferrari did not finish 1/2 in 2022.

But Mercedes and Red Bull [did](https://www.lightsoutblog.com/2022/11/16/the-last-1-2-finish-for-every-team-on-the-f1-grid/)!

This is another example of our universe producing stranger results than expected (and why we need the student-t distribution!)